# Pràctica Final Data Science - Curs 2019/2020. Product Qualified Lead

La situación que se plantea por parte de la empresa 'BetterLife' hace referencia prinicpalmente al cambio de la estratégia comercial. Véase, que en este caso se quiere reemplazar una metodología llamada Marketing Qualified Lead (MQL) por otra que lleva el nombre de Product Qualified Lead (PQL).

# Paso 1. Importaremos las librerías necesarias

In [ ]:
#Importaremos aquellas librerías que necesitaremos para realizar una lectura del dataset proporcionado por la empresa, 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Paso 2. Cargamos DataSet 

In [ ]:
# En este caso cargamos las bases de datos dado que se utilizarán a futuro para validar el modelo.
df_c = pd.read_excel("../Data/data_PQL.xlsx")

In [ ]:
#Realizaremos un primera visualización, 
df_c.head(5)

# Paso 3. Comprovamos los datos cargados
· describe
· shape
· nulls
· NaN
· dtype

In [ ]:
# Muestra un resumen de los indicadores estadisticos más usuales, los cuales utilizaremos para 
# determinar el número inicial de desviaciones estandar para eliminar si existen los outliers.
df_c.describe()

In [ ]:
# Muestra las dimenciones del dataframe
df_c.shape

In [ ]:
# Comprobamos el tipo de dato para cada columna
df_c.dtypes

In [ ]:
# Verificamos que exiten valores "null" en los campos del dataset
df_c.isnull().values.any()

In [ ]:
# Los valores "NAN" se distribuyen en los siguientes campos con estas cantidades
df_c.isnull().sum()

In [ ]:
# Eliminaremos las filas "NaN" en todos sus campos
df_c_na = df_c
df_c_na.dropna(how='all', inplace=True)

# Reemplazamos los registro "NaN" de los campos que contengan y los sustituiremos por el valor 0, 
df_c_na.fillna(0, inplace=True)

#De nuevo veremos la forma de nuestro dataframe, 
df_c_na.shape #COn este resultado podemos observar que ninguno de nuestro registro tiene todos los valores como "NaN"

In [ ]:
# Comprobamos que ya no existen valores "NaN"
df_c_na.isnull().sum()

In [ ]:
#Limpiaremos aquellas columnas que creemos que no utilizaremos para el train. Solo variables numéricas, 

df_c_clean = df_c_na.drop(["country", "industry", "job_function","job_title"], axis=1)

df_c_clean.shape

# Paso 4. Estudio de las variables de nuestro modelo y su importancia

En este cuatro paso queremos eliminar la posibilidad de que exista una situación de sobrentrenamiento de nuestro modelo. De esta forma elinaremos la posibilidad de que nuestro modelo no sirva para futuras predicciones dado que no habrá aprendido nada de forma generalizada. Decidimos realizar este paso dado que el dataset que nos proporciona la empresa tiene más columnas que filas y deberemos determinar a través del Feature Selection qué columnas tendrán una mayor importancia. 

In [ ]:
# A continaución definiremos aquellas variables independientes para nuestro posterior modelo de entrenamiento y a varibale target, 

X = df_c_clean.iloc[:,0:302]  
df_y = df_c_clean.iloc[:,-2]    

In [ ]:
#Verificaremos la importancia y el peso de cada una de las variables que compenen nuestro DataFrame, 
#Hemos utilizado el método Tree-based - SelectFromModel

from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=302)
embeded_rf_selector.fit(X, df_y)

embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = X.loc[:,embeded_rf_support].columns.tolist()
print(str(len(embeded_rf_feature)), 'selected features')

In [ ]:
#Realizaremos una vista de aquellas características que representarán mejor nustro modelo, 
df_X = embeded_rf_feature
df_X

In [ ]:
#Reduciremos nuestro modelo de train para que solo coja aquellas features que hemos definido con mayor improtancia, 
X_f = df_c_clean[['starts_2day',
 'starts_4day',
 'starts_6day',
 'starts_8day',
 'starts_10day',
 'starts_12day',
 'starts_14day',
 'starts_16day',
 'starts_18day',
 'starts_20day',
 'starts_22day',
 'starts_24day',
 'starts_26day',
 'starts_28day',
 'starts_30day',
 'starts_4day_Delta',
 'starts_6day_Delta',
 'starts_8day_Delta',
 'starts_10day_Delta',
 'starts_12day_Delta',
 'starts_14day_Delta',
 'starts_16day_Delta',
 'starts_18day_Delta',
 'starts_20day_Delta',
 'starts_22day_Delta',
 'starts_24day_Delta',
 'starts_26day_Delta',
 'starts_28day_Delta',
 'starts_30day_Delta',
 'FormC',
 'VisitC',
 'WebinarC',
 'EventC',
 'Form HandlerC',
 'Landing PageC',
 'Page ViewC',
 'Chat TranscriptC',
 'Form TrackerC',
 'countsum_retrieve',
 'countsum_apply_model',
 'countsum_filter_examples',
 'countsum_join',
 'countsum_parallel_decision_tree',
 'countsum_performance',
 'countsum_read_data_operators',
 'countsum_select_attributes',
 'countsum_set_role',
 'countsum_x_validation',
 'countsum_decision_tree',
 'countsum_multiply',
 'countsum_generate_attributes',
 'countsum_sort',
 'countsum_store',
 'countsum_EXECUTE',
 'countsum_FAILURE',
 'countsum_USER_ERROR',
 'failure_ratio',
 'user_error_ratio',
 'stopped_ratio']]

# Paso 5. Entrenamiento de los datos

Dado que ahora ya contamos con un rango de valores para analizar que usuarios comprarán o no nuestra beta, debemos preocuparnos por todo el resto de los campos para que el modelo pueda procesarlos.

In [ ]:
#Importaremos aquellas librerías necesarias para este proceso, 
from sklearn.model_selection import train_test_split

In [ ]:
#Dentro de nuestra variable a predecir reemplazaremos los valores True y False por los valores binarios 0 y 1. Dado la naturaleza del modelo que utilizaremos durante la predicción, 
y = df_y.replace(True, 1).replace(False, 0)

In [ ]:
#Realizaremos el split de nuestro dataset, 
X_f_train, X_f_test, y_train, y_test = train_test_split(X_f, y, test_size=0.30, random_state=42)    

# Paso 6. Aplicación del modelo supervisado de clasificación 

Dada la naturaleza de la predicción que queremos realizar utilizaremos el modelo de Random Forest Classifier. 

# Paso 6.1. Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Aplicamos un bucle para el RFC a cada cluster, en cada iteración asignamos 
# como valor 1 al label cuando se analiza el clúster especifico y un 0 para el resto de la información
forest = RandomForestClassifier(n_estimators=500, random_state=0, max_depth = None, 
                                 min_samples_split = 2, min_samples_leaf = 1, n_jobs=5)
 # Entrenamiento de lo modelo
model_rfc = forest.fit(X_f_train.values,y_train.values)

In [ ]:
#En este bloque podemos obervar un ranking en las variables utilizadas
variablesimportantes = model_rfc.feature_importances_
model_rfc_var_import = pd.DataFrame(list(zip(X_f_train.columns,variablesimportantes))).sort_values(1,ascending=False)
model_rfc_var_import.columns = ["Variable", "Ranking_RFC"]
model_rfc_var_import.reset_index(drop=True, inplace=True)
#model_rfc_var_import.Variable.values.tolist()
model_rfc_var_import

In [ ]:
#A continuación realizaremos la aplicación del modelo de RFC, 
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report

y_train_pred = model_rfc.predict(X_f_train)
y_test_pred = model_rfc.predict(X_f_test)

print("Accurancy de train es: ",accuracy_score(y_train, y_train_pred))
print("Accurancy de test es: ",accuracy_score(y_test, y_test_pred))

print(classification_report(y_test, y_test_pred))

Como podemos observar con las transformaciones realizadas y la posterior aplicación del modelo Random Forest Classifier, obtenemos un accuracy del train 98,9% y un accuracy del test del 62,3%.  

# Paso 6.1.1 - Aplicación de Logistic Regression

Dada la situación de la predicción que queremos realizar también utilizaremos el modelo de Logistic Regression para poder realizar una comparación entre ambos modelos y así escojer aquél que no proporciones mejores predicciones. 

In [ ]:
from sklearn.linear_model import LogisticRegression

# Aplicación del Modelo logistico para el dataset, 
model_log = LogisticRegression()
model_log.fit(X_f_train,y_train)
# Predicciones y Accurancy del Modelo Logistico
y_train_pred_log = model_log.predict(X_f_train)
y_test_pred_log = model_log.predict(X_f_test)

print("Accurancy de train es: ",accuracy_score(y_train, y_train_pred_log))
print("Accurancy de test es: ",accuracy_score(y_test, y_test_pred_log))

print(classification_report(y_test, y_test_pred_log))

Como podemos observar con las transformaciones realizadas y la posterior aplicación del modelo Logistic Regression, obtenemos un accuracy del train 75,1% y un accuracy del test del 67,1%.

# Paso 7. Cargaremos y realizaremos las transformaciones correspondientes al fichero de predicción

In [ ]:
#Véase a continuación que haremos una lectura del fichero de predict que nos proporciona la companyía, 
df_c_pred = pd.read_excel("../Data/data_PQLpredict.xlsx")
df_c_pred

# Paso 7.1. Transformación del modelo

A continuación realizaremos el mismo proceso de transformación de los datos que hemos llevado a cabo con el data set que nos ha servido para realizar nuestro entrenamiento

In [ ]:
# Muestra un resumen de los indicadores estadisticos más usuales, los cuales utilizaremos para 
# determinar el número inicial de desviaciones estandar para eliminar si existen los outliers.
df_c_pred.describe()

In [ ]:
# Muestra las dimenciones del dataframe
df_c_pred.shape

In [ ]:
# Verificamos que exiten valores "null" en los campos del dataset
df_c_pred.isnull().values.any()

In [ ]:
# Los valores "NAN" se distribuyen en los siguientes campos con estas cantidades
df_c_pred.isnull().sum()

In [ ]:
# Eliminaremos las filas "NaN" en todos sus campos
df_c_na_pred = df_c_pred
df_c_na_pred.dropna(how='all', inplace=True)

# Reemplazamos los registro "NaN" de los campos que contengan y los sustituiremos por el valor 0, 
df_c_na_pred.fillna(0, inplace=True)

#De nuevo veremos la forma de nuestro dataframe, 
df_c_na_pred.shape #COn este resultado podemos observar que ninguno de nuestro registro tiene todos los valores como "NaN"

In [ ]:
# Comprobamos que ya no existen valores "NaN"
df_c_na_pred.isnull().sum()

In [ ]:
#Limpiaremos aquellas columnas que creemos que no utilizaremos para el train. Solo variables numéricas, 
df_c_clean_pred = df_c_na_pred.drop(["country", "industry", "job_function","job_title"], axis=1)
df_c_clean_pred.shape

In [ ]:
#Definiremos de nuevo para nuestro DataSet de predicción aquellas características que definirán la predicción,
X_f_pred = df_c_clean_pred[['starts_2day',
 'starts_4day',
 'starts_6day',
 'starts_8day',
 'starts_10day',
 'starts_12day',
 'starts_14day',
 'starts_16day',
 'starts_18day',
 'starts_20day',
 'starts_22day',
 'starts_24day',
 'starts_26day',
 'starts_28day',
 'starts_30day',
 'starts_4day_Delta',
 'starts_6day_Delta',
 'starts_8day_Delta',
 'starts_10day_Delta',
 'starts_12day_Delta',
 'starts_14day_Delta',
 'starts_16day_Delta',
 'starts_18day_Delta',
 'starts_20day_Delta',
 'starts_22day_Delta',
 'starts_24day_Delta',
 'starts_26day_Delta',
 'starts_28day_Delta',
 'starts_30day_Delta',
 'FormC',
 'VisitC',
 'WebinarC',
 'EventC',
 'Form HandlerC',
 'Landing PageC',
 'Page ViewC',
 'Chat TranscriptC',
 'Form TrackerC',
 'countsum_retrieve',
 'countsum_apply_model',
 'countsum_filter_examples',
 'countsum_join',
 'countsum_parallel_decision_tree',
 'countsum_performance',
 'countsum_read_data_operators',
 'countsum_select_attributes',
 'countsum_set_role',
 'countsum_x_validation',
 'countsum_decision_tree',
 'countsum_multiply',
 'countsum_generate_attributes',
 'countsum_sort',
 'countsum_store',
 'countsum_EXECUTE',
 'countsum_FAILURE',
 'countsum_USER_ERROR',
 'failure_ratio',
 'user_error_ratio',
 'stopped_ratio']]

# Paso 8. Valoración del campo objetivo mediante el modelo Logistic Regression

Una vez realizadas todas las transformaciones adecuadas para el tratamiento del nuevo dataset, aplicaremos el modelo que hemos entrenado anteriormente para poder predecir el comportamiento de los usuarios que aparecen en el fichero data_PQLpredict, proporcionado por la empresa. 

In [ ]:
#A partir de los datos que obtenemos de la empresa parar realizar la predicción y utilizando el modelo de entreno,
#aplicaremos este mismo modelo entrenado sobre los datos de predicción. Obtendremos el siguiente output, 

df_predict = pd.DataFrame(model_log.predict(X_f_pred))
df_predict

#Transformaremos el df_final. Reemplazaremos los valor 1 y 0 por True y False, así obtendremos un un mejor visión de la predicción, 
df_predict_etl = df_predict.replace(1.0, True).replace(0.0, False)
df_predict_etl

#Renombraremos la columna output de la predicción, 
df_predict_etl.rename(columns={0: 'Test_Prediction_PQL'}, inplace=True)
df_predict_etl

In [ ]:
#Realizaremos un concat del dataframe que continene los datos de la predicción con el dataset que obtenemos para predecir, 
df_final = pd.concat([df_c_pred, df_predict_etl], axis=1)
df_final

In [ ]:
#Para poder obtener un output que tenga una mejor estructura solo cojeremos aquellos valores que corresponde a variables 
#descriptiva y al outout obtenido de la predicción: 

df_final_sel = df_final[["country", "industry", "job_function","job_title","Test_Prediction_PQL"]]
df_final_sel

In [ ]:
#Crear fichero excel que queremos visualizar en excel y que posteriormente utilizaremos para nuestro análisis,  
df_final_sel.to_excel('../Data/Test_Prediction_PQL.xlsx')

In [ ]:
#STOP